# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from urllib.parse import parse_qs, urlencode, urlparse
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from copy import deepcopy
import numpy as np
from sklearn.model_selection import train_test_split
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers, callbacks
from tensorflow.keras.models import load_model

In [2]:
thre = 0.8
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None, batch_size=1000, epochs=25, test_size=0.10):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.hangman_url = self.determine_hangman_url()
        self.guessed_letters = []
        full_dictionary_location = "/kaggle/input/data-of-trexquant/words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.n_grams = self.build_n_gram(self.full_dictionary, max_n=5)
        self.max_length = 29
        self.test_size = test_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.train_model() 
        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']
        data = {link: 0 for link in links}

        for link in links:
            requests.get(link)
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def build_n_gram(self, word_list, max_n):
        n_grams = {}
        for n in range(1, max_n + 1):
            n_grams[n] = collections.defaultdict(int)
            for word in word_list:
                for i in range(len(word) - n + 1):
                    n_grams[n][word[i:i + n]] += 1
        return n_grams

    def get_n_gram_prob(self, word, guessed_letters):
        return self.calculate_n_gram_prob(self.n_grams, word.replace('_', '.'), guessed_letters)

    def calculate_n_gram_prob(self, n_grams, word, guessed_letters):
        probabilities = {}
        for n in n_grams:
            for i in range(len(word) - n + 1):
                gram = word[i:i + n]
                if gram in n_grams[n]:
                    next_chars = n_grams[n][gram]
                    if isinstance(next_chars, dict):
                        total_count = sum(next_chars.values())
                        prob_distribution = {char: count / total_count for char, count in next_chars.items() if char not in guessed_letters}
                        probabilities.update(prob_distribution)
        return probabilities

    def guess(self, word):
        clean_word = word[::2]
        encoded_word = self.encode_word(clean_word)
        paded_word = pad_sequences([encoded_word], maxlen=self.max_length, padding='post')
        pred = self.model.predict(paded_word)[0]

        lstm_prob = {chr(i + ord('a')): pred[i] for i in range(26)}
        ngram_prob = self.get_n_gram_prob(clean_word, self.guessed_letters)

        combined_prob = {}
        lstm_weight = 0.75
        ngram_weight = 0.25

        for char in lstm_prob:
            combined_prob[char] = lstm_weight * lstm_prob[char] + ngram_weight * ngram_prob.get(char, 0)

        sorted_chars = sorted(combined_prob.items(), key=lambda x: x[1], reverse=True)

        for letter, _ in sorted_chars:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
        return guess_letter

    def encode_word(self, word):
        encoded_word = []
        for cha in word:
            if ord(cha) == ord("_"):
                encoded_word.append(27)
            else:
                encoded_word.append(ord(cha) - ord("a"))
        return encoded_word

    def get_xy(self):
        encoded_dictionary = [self.encode_word(word) for word in self.full_dictionary]
        X, y = [], []

        for encoded_word in encoded_dictionary:
            dic = {cha: [] for cha in set(encoded_word)}
            for i, cha in enumerate(encoded_word):
                dic[cha].append(i)

            for cha, lis in dic.items():
                masked_word = deepcopy(encoded_word)
                for pos in lis:
                    masked_word[pos] = 27
                X.append(masked_word + [0] + [0])
                y.append(cha)

        X = pad_sequences(X, maxlen=self.max_length, padding='post')
        y = to_categorical(y, num_classes=26)
        train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=self.test_size, random_state=42)
        return train_x, test_x, train_y, test_y

    def train_model(self):
        (train_x, test_x, train_y, test_y) = self.get_xy()

        self.model = Sequential()
        self.model.add(Embedding(input_dim=28, output_dim=128, input_length=self.max_length))
        self.model.add(Bidirectional(LSTM(64, return_sequences=True,  kernel_regularizer=regularizers.l2(0.00001))))
        self.model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.00001))))
        self.model.add(Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=regularizers.l2(0.00001))))
        self.model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=regularizers.l2(0.00001))))
        self.model.add(Bidirectional(LSTM(64)))
        self.model.add(Dense(26, activation='softmax'))

        self.model.compile(optimizer="adam", 
                           loss="categorical_crossentropy", 
                           metrics=['accuracy', 'top_k_categorical_accuracy'])
        lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                                   factor=0.2, 
                                                   patience=1, 
                                                   min_lr=1e-6, 
                                                   min_delta=0.005)

        checkpoint_filepath = '/tmp/ckpt/checkpoint.model.keras'
        model_checkpoint_callback = callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', mode='max', save_best_only=True)

        early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.005, restore_best_weights=True)
        
        already_trained = True
        if already_trained:
            self.model = load_model('/kaggle/input/trexquant/dc_model.keras')
            validation_loss, validation_accuracy, top_k_accuracy = self.model.evaluate(test_x, test_y)
            print(f"Validation Accuracy: {validation_accuracy:.4f}")

        else:
            history = self.model.fit(train_x, train_y,
                                     batch_size=self.batch_size,
                                     epochs=self.epochs,
                                     validation_data=(test_x, test_y),
                                     callbacks=[early_stopping, lr_scheduler, model_checkpoint_callback])
            self.model.save('/kaggle/working/dc_model.keras')
        return

    def get_order(self, prediction):
        dic = {i:prediction[i] for i in range(len(prediction))}
        sorted_cha_list = [chr(index + ord("a")) for index, value in sorted(dic.items(), key=lambda x:x[1],reverse = True)]
        return sorted_cha_list

        for letter in pred:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
        return guess_letter
    
    


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"
        if self.access_token:
        
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [3]:
api = HangmanAPI(access_token="4459375e913f4cdbe4456df831c188", timeout = 2000, batch_size = 128, epochs = 20, test_size = 0.1)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
11821/11821 ━━━━━━━━━━━━━━━━━━━━ 679s 57ms/step - accuracy: 0.4014 - loss: 1.9148 - top_k_categorical_accuracy: 0.7699 - val_accuracy: 0.6260 - val_loss: 1.1955 - val_top_k_categorical_accuracy: 0.9185 - learning_rate: 0.0010
Epoch 2/20
11821/11821 ━━━━━━━━━━━━━━━━━━━━ 675s 57ms/step - accuracy: 0.6471 - loss: 1.1416 - top_k_categorical_accuracy: 0.9251 - val_accuracy: 0.6847 - val_loss: 1.0476 - val_top_k_categorical_accuracy: 0.9377 - learning_rate: 0.0010
Epoch 3/20
11821/11821 ━━━━━━━━━━━━━━━━━━━━ 707s 60ms/step - accuracy: 0.7013 - loss: 0.9980 - top_k_categorical_accuracy: 0.9425 - val_accuracy: 0.7103 - val_loss: 0.9877 - val_top_k_categorical_accuracy: 0.9435 - learning_rate: 0.0010
Epoch 4/20
11821/11821 ━━━━━━━━━━━━━━━━━━━━ 724s 61ms/step - accuracy: 0.7287 - loss: 0.9273 - top_k_categorical_accuracy: 0.9499 - val_accuracy: 0.7213 - val_loss: 0.9639 - val_top_k_categorical_accuracy: 0.9460 - learning_rate: 0.0010
Epoch 5/20
11821/11821 ━━━━━━━━━━━━━━━━━━━━ 712s 60m

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [4]:
c=0
[_,_,_,pre] = api.my_status() # Get my game stats: (# of tries, # of wins)
for i in range (150) :
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    if total_practice_successes>pre:
        c+=1
        pre = total_practice_successes
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully start a new game! Game ID: 7940f1c25667. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ _ .
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 798ms/step
Guessing letter: a
Sever response: {'game_id': '7940f1c25667', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Guessing letter: e
Sever response: {'game_id': '7940f1c25667', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ e _ _ _ _ _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Guessing letter: l
Sever response: {'game_id': '7940f1c25667', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ _ _ e _ _ _ _ _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Guessing letter: m
Sever response: {'game_id': '7940f1c25667', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _ _ _ _ e _ _ _ _ _ _ _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Guessing letter: o
Sever response: {'game_id': '7940f1c25667', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ _ _

In [5]:
c

60

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [6]:
# for i in range(1000):
#     print('Playing ', i, ' th game')
#     # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
#     #api.start_game(practice=0,verbose=False)
    
#     # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
#     time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [7]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

ZeroDivisionError: division by zero